In [7]:
#Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [8]:
# Import the data for the employee emails
emp_emails = pd.read_excel("Email Address.xlsx", sheet_name=1)

# Rename columns name
emp_emails.rename(columns = {"Email Address" : "emails" }, inplace=True)

# Extract the username from email by removing domain
emp_emails['username'] = emp_emails['emails'].str.replace('@xyz.com', '')
emp_emails.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  


,emails,username
0,nishikachhabra99@xyz.com,nishikachhabra99
1,aiswarya@xyz.com,aiswarya
2,rachna.prasad@xyz.com,rachna.prasad
3,rishi.jha@xyz.com,rishi.jha
4,tanyakumbhat@xyz.com,tanyakumbhat


In [9]:
# Import the data for the employee ID and names
emp_names = pd.read_excel("Email Address.xlsx", sheet_name=0)
emp_names.head()

,ID,Name
0,572,Sharad Agarwal
1,393,Rahul Sankhe
2,638,Sampath Vedantam
3,533,Varun Mahajan
4,633,Nimish Sharma


In [10]:
# Validate the number of records in both the dataframes are same.
# Required to clear any duplicates or missing entries.
print(emp_names.shape[0], emp_emails.shape[0])

559 559


Using the FuzzyWuzzy package in Pandas library, we can estimate the connection between employee names and emails.

FuzzyWuzzy is a library of Python which is used for string matching. Fuzzy string matching is the process of finding strings that match a given pattern.

Reference: https://www.geeksforgeeks.org/fuzzywuzzy-python-library/

In [11]:
# pip install fuzzywuzzy

In [12]:
from fuzzywuzzy import process, fuzz

/usr/local/lib/python3.7/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [13]:
actual_email = []
similarity = []

# We will check the similarity between text string of the emp_names and emp_emails dataframes.
# For each record in emp_names, find the similarity score of a record with corresponding entry in emp_emails dataframe
for i in emp_names.Name:
    
  if pd.isnull( i ) :          
    actual_email.append(np.null)
    similarity.append(np.null)     
  else :
    # The process functions from fuzzywuzzy library package does the calculation by using Lavenshtein's Distance
    ratio = process.extract(i, emp_emails.username, limit=1, scorer= fuzz.partial_ratio)

    # Append the values of username matched and similarity score calculated into the lists below
    actual_email.append(ratio[0][0])
    similarity.append(ratio[0][1]) 

# Add back the domain name to username and append the similarity and generated email addresses to a dataframe
compare = emp_names.copy()
compare['actual_email'] = pd.Series(actual_email)

compare['actual_email'] = compare['actual_email'] + '@xyz.com'
compare['similarity'] = pd.Series(similarity)

In [14]:
# For clarity, only take relevant columns from the dataframe
final_result = compare[['Name', 'actual_email', 'similarity']]
final_result.head()

,Name,actual_email,similarity
0,Sharad Agarwal,sagar@xyz.com,80
1,Rahul Sankhe,rahul@xyz.com,100
2,Sampath Vedantam,sampath.vedantam@xyz.com,100
3,Varun Mahajan,varun@xyz.com,100
4,Nimish Sharma,nimish@xyz.com,100


In [15]:
final_result = final_result.sort_values('similarity')
final_result.head(150).style.background_gradient(subset='similarity',
                                               cmap='summer_r')

,Name,actual_email,similarity
405,Dj Khenzy,subhadip.mukherjee@xyz.com,56
512,Vaghela Dhaval,madhu@xyz.com,60
357,asdfad asdfasdf,hasan@xyz.com,60
506,Jake Long,lkanga@xyz.com,60
154,SYEDALAY I NAZEEF,navdeep@xyz.com,62
266,Vinay Reddy,vijayan.rect@xyz.com,64
262,Freaquer Red,ram@xyz.com,67
36,Mogomotsi Mazunga,lkanga@xyz.com,67
96,Ryu Base,sneh@xyz.com,67
419,Jublee Singh Rajput,jobanjeet.singh@xyz.com,67


Number of matches for different similarity scores
Find accuracy of the match

In [16]:
check = pd.DataFrame()

# Segregate the dataframe with matches which have high similarity (>=90), medium similarity (between 75 and 90) and low similarity (below 75)
check['high'] = final_result.apply(lambda x : True if x['similarity'] >= 90 else False, axis = 1)
check['medium'] = final_result.apply(lambda x : True
            if x['similarity'] < 90 and x['similarity'] >= 75 else False, axis = 1)
check['low'] = final_result.apply(lambda x : True if x['similarity'] < 75 else False, axis = 1)

# Count number of True in the series
high = len(check['high'][check['high'] == True].index)
med = len(check['medium'][check['medium'] == True].index)
low = len(check['low'][check['low'] == True].index)

# We can verify that the sum of high, med and low matches equals the total entries in the data
print(high, med, low)
print(emp_emails.shape[0])

# Displaying the percentage of each match
print("% of high matches: ", high/emp_emails.shape[0])
print("% of medium matches: ", med/emp_emails.shape[0])
print("% of low matches: ", low/emp_emails.shape[0])

472 66 21
559
% of high matches:  0.8443649373881932
% of medium matches:  0.11806797853309481
% of low matches:  0.03756708407871199


Convert the final result dataframe to excel file

In [27]:
export = final_result.loc[:, ['Name','actual_email']]
export.to_excel('output.xlsx', index=False)